In [5]:
import bs4 as bs
import requests

In [53]:
base_url = "https://www.digischool.fr"

In [1]:
url = "https://www.digischool.fr/college/troisieme/histoire-geographie-emc"

In [2]:
def get_soup(url):
    response = requests.get(url)
    soup = bs.BeautifulSoup(response.text, 'html.parser')
    return soup

In [59]:
def get_links(url):
    
    soup = get_soup(url)
    if soup is None:
        print("Error: No soup")
    else:
        course_links = [
        a["href"]
        for a in soup.find_all("a", href=True)
        if a["href"].startswith("/cours/") and "quiz" not in a["href"]
    ]
    titles = [
    p.get_text(strip=True)
    for p in soup.find_all("p")
    if p.has_attr("class") and any("body-s-16-bold" in cls for cls in p["class"])
    ]

  
    h2s = [
    h2.get_text(strip=True)
    for h2 in soup.find_all("h2")
    if h2.has_attr("class") and any("body-s-16-bold" in cls for cls in h2["class"])
    ]
   
    course_links = [
    (title, base_url + link,)
    for title, link, h2 in zip(titles, course_links, h2s)
    ]
        
    return course_links


In [60]:
course_links = get_links(url)

In [61]:
course_links

[('L’expérience combattante pendant la\xa0Première Guerre mondiale',
  'https://www.digischool.fr/cours/l-experience-combattante-pendant-la-premiere-guerre-mondiale?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-1945'),
 ('Les civils dans une guerre totale',
  'https://www.digischool.fr/cours/les-civils-dans-une-guerre-totale?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-1945'),
 ('De la révolution bolchevique au\xa0régime totalitaire stalinien',
  'https://www.digischool.fr/cours/de-la-revolution-bolchevique-au-regime-totalitaire-stalinien?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-1945'),
 ('Le régime totalitaire nazi',
  'https://www.digischool.fr/cours/le-regime-totalitaire-nazi?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-

In [20]:
def get_course(url):
    soup = get_soup(url)
    if not soup:
        print("Error: No soup")
        return {}

    courses = {}
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("/cours/") and "quiz" not in href:
            # Récupération du titre contenu dans une balise <p> avec la classe attendue
            title_elem = a.find("p", class_=lambda cls: cls and "body-s-16-bold" in cls)
            title = title_elem.get_text(strip=True) if title_elem else ""

            # Récupérer l'élément H2 le plus proche en amont avec une classe contenant "body-s-16-bold"
            h2_elem = a.find_previous("h2", class_=lambda cls: cls and "body-s-16-bold" in cls)
            category = h2_elem.get_text(strip=True) if h2_elem else ""

            courses[href] = {"title": title, "category": category}
            # remove duplicate courses
            

    return courses

In [21]:
courses = get_course(url)
courses

{'/cours/l-experience-combattante-pendant-la-premiere-guerre-mondiale?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-1945': {'title': 'L’expérience combattante pendant la\xa0Première Guerre mondiale',
  'category': "Histoire - L'Europe, un théâtre majeur des guerres totales (1914-1945)"},
 '/cours/les-civils-dans-une-guerre-totale?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-1945': {'title': 'Les civils dans une guerre totale',
  'category': "Histoire - L'Europe, un théâtre majeur des guerres totales (1914-1945)"},
 '/cours/de-la-revolution-bolchevique-au-regime-totalitaire-stalinien?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-1945': {'title': 'De la révolution bolchevique au\xa0régime totalitaire stalinien',
  'category': "Histoire - L'Europe, un théâtre majeur des guerres totales (1914-1945

In [22]:
import re
def get_lessons(url):
    # function to retrieve the lessons contente from the url
    soup = get_soup(url)
    if not soup:
        print("Error: No soup")
    latex_span = soup.find('span', class_="__Latex__")
    if latex_span:
            article = latex_span.find_parent('article')
            if article:
                content = article.get_text(separator=' ', strip=True)
    
    return content
      

In [23]:
url_lesson = "https://www.digischool.fr/cours/les-civils-dans-une-guerre-totale?srcpath=%2Cfr%2Csuperbrevet%2Ctroisieme%2Chistoire-geographie-emc%2Ceurope-theatre-majeur-guerres-totales-1914-1945"
soup = get_soup(url_lesson)

In [24]:
content = get_lessons(url_lesson)

In [25]:
print(content)

Légende de la leçon Jaune : dates, chiffres. Vert : définitions. Introduction Quelle est la place des civils dans la Grande Guerre ? I. La mobilisation de l’arrière dans l’effort de guerre 1) Une mobilisation économique et financière La guerre totale suppose une mobilisation de tous : les civils, notamment les femmes, remplacent les hommes partis au front, dans tous les secteurs d’activité. Les matières premières et les produits agricoles sont réquisitionnés pour l’armée. Mot-clé L’inflation est la hausse généralisée des prix. L’État doit financer l’effort de guerre : il lance donc des emprunts d’État , auprès des États-Unis notamment, crée de nouveaux impôts directs (en France, sur le revenu en 1917 ) ou indirects (loterie nationale). Il augmente la masse monétaire en circulation en émettant de nouveaux billets, entraînant une forte inflation. Les États favorisent le développement des industries de guerre qui produisent massivement de nouvelles armes, comme les avions et les chars. 2)

In [26]:
def run():
    info = {}
    # get the courses links
    courses = get_links(url)
    # get the lessons content
    for course in courses:
        content = get_lessons(base_url + course)
        info[course] = content
    return info

        

In [27]:
info = run()
info

TypeError: can only concatenate str (not "tuple") to str

In [28]:
test_kartable = "https://www.kartable.fr/ressources/histoire/cours/les-civils-et-les-militaires-dans-la-premiere-guerre-mondiale/60392"

In [29]:
soup = get_soup(test_kartable)

In [30]:
url_kar_prog ="https://www.kartable.fr/troisieme/histoire/specifique"

In [31]:
soup = get_soup(url_kar_prog)

In [32]:
from bs4 import BeautifulSoup

def get_all_theme_courses(url: str) -> list:
    soup = get_soup(url)
    themes_dict = {}
    
    sections = soup.find_all("section", class_=lambda c: c and "categories__group" in c)
    for section in sections:
        theme_div = section.find("div", class_=lambda c: c and "categories__group-type" in c)
        if theme_div:
            name_elem = theme_div.find("p", class_=lambda c: c and "subtitle-1" in c)
            title_elem = theme_div.find("h3")
            theme_name = name_elem.get_text(strip=True) if name_elem else ""
            theme_title = title_elem.get_text(strip=True) if title_elem else ""
        else:
            theme_name, theme_title = "", ""
        
        key = (theme_name, theme_title)
        courses = []
        # Restreindre la recherche aux catégories hiérarchiques en enfant direct
        hierarchy = section.find("categories-hierarchy", recursive=False)
        if hierarchy:
            articles = hierarchy.find_all("article", class_=lambda c: c and "category" in c)
            for article in articles:
                a_tag = article.find("a", class_=lambda c: c and "category__link" in c)
                if a_tag:
                    link = a_tag.get("href", "")
                    content = a_tag.find("article", class_=lambda c: c and "category__content" in c)
                    if content:
                        title_elem = content.find("p", class_=lambda c: c and "category__title" in c)
                        course_title = title_elem.get_text(strip=True) if title_elem else ""
                        courses.append({"link": link, "title": course_title})
        
        if key in themes_dict:
            themes_dict[key]["courses"].extend(courses)
        else:
            themes_dict[key] = {
                "theme_name": theme_name,
                "theme_title": theme_title,
                "courses": courses
            }
    
    return list(themes_dict.values())

In [33]:
themes = get_all_theme_courses(url_kar_prog)

In [34]:
themes

[{'theme_name': 'Thème 1',
  'theme_title': "L'Europe, un théâtre majeur des guerres totales (1914-1945)",
  'courses': [{'link': '/ressources/histoire/cours/les-civils-et-les-militaires-dans-la-premiere-guerre-mondiale/60392',
    'title': 'Les civils et les militaires dans la Première Guerre mondiale'},
   {'link': '/ressources/histoire/cours/les-democraties-fragilisees-et-les-experiences-totalitaires-dans-leurope-de-lentre-deux-guerres/60393',
    'title': "Les démocraties fragilisées et les expériences totalitaires dans l'Europe de l’entre-deux-guerres"},
   {'link': '/ressources/histoire/cours/la-seconde-guerre-mondiale-une-guerre-daneantissement-2/60394',
    'title': "La Seconde Guerre mondiale : une guerre d'anéantissement"},
   {'link': '/ressources/histoire/cours/la-france-defaite-et-occupee-collaboration-et-resistance/60395',
    'title': 'La France défaite et occupée : collaboration et résistance'},
   {'link': '/ressources/histoire/cours/independances-et-construction-de-no

In [35]:
test_schoolmovze = "https://www.schoolmouv.fr/cours/la-seconde-guerre-mondiale-une-guerre-d-aneantissement/fiche-de-cours"

In [36]:
soup = get_soup(test_schoolmovze)

In [37]:
content_schoolmovze = soup.find('div', class_="luna-content_luna-content__section__z7rg5").text

In [38]:
print(content_schoolmovze)

Introduction :
La Seconde Guerre mondiale (1939-1945) se caractérise par l’idée d’anéantissement total de l’ennemi. C’est une guerre d’une violence extrême qui met en place une politique d’extermination systématique et organisée de l’ennemi.
Pourquoi peut-on qualifier la Seconde Guerre mondiale de guerre d’anéantissement ? Pour y répondre, nous étudierons trois grandes parties : la première partie évaluera les dimensions planétaires du conflit et proposera une brève chronologie. L’exemple de la bataille de Stalingrad, la deuxième partie, permettra d’étudier de quelles manières l’intégralité des forces matérielles et morales des peuples ont été utilisées. Le thème du génocide des Juifs et des Tsiganes formera une troisième partie. Nous soulignerons ici que l’objectif des nazis était bien une destruction totale et systématique de l’ennemi.
Un affrontement aux dimensions planétaires
1939-1942 : les victoires de l’Axe
Quelques petites définitions sont nécessaires avant de commencer cette p

In [42]:
url_prog_school= "https://www.schoolmouv.fr/3eme/histoire"

In [43]:
soup = get_soup(url_prog_school)

In [44]:

def get_courses_from_html(url: str) -> dict:
    soup = get_soup(url)
    container = soup.find("div", class_="program-template_program__content__w3oIs")
    chapters = {}
    if not container:
        return chapters

    chapter_divs = container.find_all("div", class_="course-list_chapter__13m89")
    for chapter in chapter_divs:
        # Récupérer le titre du chapitre depuis le h2
        header = chapter.find("div", class_="course-list_chapter__title--without-cups__dehJf")
        h2 = header.find("h2") if header else None
        chapter_title = h2.get_text(strip=True) if h2 else "Sans titre"

        courses_list = []
        ul = chapter.find("ul", class_="course-list_chapter__course-list__x_5Q6")
        if ul:
            for li in ul.find_all("li", class_="course-list_chapter__course-list--item-without-cups__CJUPl"):
                a = li.find("a", class_="course-title-card-module_course-title-card__YrMOq")
                if a:
                    link = a.get("href")
                    p = a.find("p")
                    course_title = p.get_text(strip=True) if p else ""
                    courses_list.append({
                        "link": link,
                        "title": course_title
                    })
        chapters[chapter_title] = courses_list

    return chapters

In [45]:
chapters = get_courses_from_html(url_prog_school)

In [47]:
chapters

{"L'Europe, un théâtre majeur des guerres totales (1914-1945)": [{'link': 'https://www.schoolmouv.fr/cours/la-premiere-guerre-mondiale-vers-une-guerre-totale/fiche-de-cours',
   'title': 'La Première Guerre mondiale : vers une guerre totale'},
  {'link': 'https://www.schoolmouv.fr/cours/les-regimes-totalitaires-dans-les-annees-1930/fiche-de-cours',
   'title': 'Les régimes totalitaires dans les années 1930'},
  {'link': 'https://www.schoolmouv.fr/cours/la-france-durant-lentre-deux-guerres/fiche-de-cours',
   'title': "La France durant l'entre-deux guerres"},
  {'link': 'https://www.schoolmouv.fr/cours/la-seconde-guerre-mondiale-une-guerre-d-aneantissement/fiche-de-cours',
   'title': 'La Seconde Guerre mondiale : une guerre d’anéantissement'},
  {'link': 'https://www.schoolmouv.fr/cours/la-france-de-vichy-1940-1944/fiche-de-cours',
   'title': 'La France de Vichy (1940-1944)'}],
 'Le monde depuis 1945': [{'link': 'https://www.schoolmouv.fr/cours/la-decolonisation/fiche-de-cours',
   't

In [46]:
# Print the keys of the chapters dictionary
print(chapters.keys())

# Access the first key to get the corresponding value
first_key = list(chapters.keys())[0]
chapters[first_key]

dict_keys(["L'Europe, un théâtre majeur des guerres totales (1914-1945)", 'Le monde depuis 1945', 'Françaises et Français dans une République repensée'])


[{'link': 'https://www.schoolmouv.fr/cours/la-premiere-guerre-mondiale-vers-une-guerre-totale/fiche-de-cours',
  'title': 'La Première Guerre mondiale : vers une guerre totale'},
 {'link': 'https://www.schoolmouv.fr/cours/les-regimes-totalitaires-dans-les-annees-1930/fiche-de-cours',
  'title': 'Les régimes totalitaires dans les années 1930'},
 {'link': 'https://www.schoolmouv.fr/cours/la-france-durant-lentre-deux-guerres/fiche-de-cours',
  'title': "La France durant l'entre-deux guerres"},
 {'link': 'https://www.schoolmouv.fr/cours/la-seconde-guerre-mondiale-une-guerre-d-aneantissement/fiche-de-cours',
  'title': 'La Seconde Guerre mondiale : une guerre d’anéantissement'},
 {'link': 'https://www.schoolmouv.fr/cours/la-france-de-vichy-1940-1944/fiche-de-cours',
  'title': 'La France de Vichy (1940-1944)'}]

In [119]:
#sve the page
with open("page.html", "w") as file:
    file.write(soup.prettify())